In [ ]:
import os
import sys

In [ ]:
def get_info_by_folder(folder_name, file_name=""):
    # --- 1. 刪除原本的 core_projects 字典，改用自動解析邏輯 ---
    # 將檔名如 "Smart_Logistics_Matrix.py" 轉為 "Smart Logistics Matrix"
    clean_name = file_name.replace(".py", "").replace("_", " ")
    auto_feature = clean_name.title() # 首字母大寫

    # --- 2. 定義各資料夾的通用學習重點 ---
    info = {
        "Project": (f"{auto_feature} 實作", "模組化架構、系統整合與實戰應用"),
        "function": ("函式封裝練習", "參數傳遞、Return 回傳值應用"),
        "loop": ("迴圈演算法練習", "for/while 迭代、控制流程"),
        "if_else": ("邏輯判斷練習", "條件分歧、布林邏輯運算"),
        "list": ("資料結構與排序", "串列操作、演算法效率優化"),
        "variable_constant": ("基礎語法練習", "變數賦值、基本數學運算"),
        "utils": ("工具程式", "專案輔助腳本與自動化工具"),
        "multidimensional_list": (f"多維串列應用 - {auto_feature}", "矩陣運算、巢狀迴圈與表格處理"),
        "file_input": ("檔案處理練習", "例外處理、檔案讀寫安全檢核") #修正
    }

    # 如果資料夾在 info 裡有定義，就回傳對應內容；否則回傳通用格式
    if folder_name in info:
        return info[folder_name]
    
    # 針對 Learning 裡面的子資料夾做一個萬用格式
    return (f"{auto_feature} 練習", "語法熟悉與邏輯實作")

In [ ]:
def process_single_file(file_path):
    """處理單一檔案的標註邏輯"""
    if not file_path.endswith(".py") or not os.path.exists(file_path):
        return

    file_name = os.path.basename(file_path)
    # 取得檔案所在的資料夾名稱
    parent_dir = os.path.dirname(os.path.abspath(file_path))
    folder_name = os.path.basename(parent_dir)
    
    # 取得功能與重點
    feature, point = get_info_by_folder(folder_name, file_name)
    docstring = f'"""\n檔名：{file_name}\n功能：{feature}\n學習重點：{point}\n"""\n'

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # 檢查是否已有舊標註並替換
    if lines and lines[0].strip().startswith('"""'):
        end_index = -1
        for i, line in enumerate(lines):
            if i > 0 and '"""' in line.strip():
                end_index = i
                break
        new_content = docstring + "".join(lines[end_index+1:]) if end_index != -1 else docstring + "".join(lines)
    else:
        new_content = docstring + "".join(lines)

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(new_content)
    print(f"✅ 已完成自動化標註: [{folder_name}] {file_name}")

In [ ]:
def update_all_folders():
    # 取得根目錄 (假設 add_docstrings.py 在 utils/ 資料夾內)
    base_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    root_targets = ["Project", "Learning"]
    
    for folder in root_targets:
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path): continue
        
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".py") and file != "add_docstrings.py":
                    process_single_file(os.path.join(root, file))

In [ ]:
if __name__ == "__main__":
    if len(sys.argv) > 1:
        target = sys.argv[1]
        if os.path.isfile(target):
            process_single_file(target)
        else:
            print(f"❌ 找不到目標: {target}")
    else:
        print("💡 執行全域更新 (掃描 Project 與 Learning)...")
        update_all_folders()